<a href="https://colab.research.google.com/github/galitneu/auto-eda-tool/blob/main/9498.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# --- שלב 0: ייבוא ספריות וחיבור לגוגל דרייב ---
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import RandomizedSearchCV

# חיבור סביבת העבודה לגוגל דרייב
from google.colab import drive
drive.mount('/content/drive')

# --- שלב 1: הגדרת נתיבים, טעינת הנתונים וסינון לפי שנה ---
DRIVE_PATH = '/content/drive/MyDrive/KaggleProject/'

try:
    df_train_raw = pd.read_csv(f'{DRIVE_PATH}Train.csv', low_memory=False)
    df_valid_raw = pd.read_csv(f'{DRIVE_PATH}Valid.csv', low_memory=False)
except FileNotFoundError:
    print(f"שגיאה: ודא שהקבצים 'Train.csv' ו-'Valid.csv' נמצאים בתיקייה: {DRIVE_PATH}")
    exit()

# --- סינון הנתונים לשנים 2008 ואילך ---
print("מבצע סינון ראשוני לשמירת נתונים משנת 2008 ואילך...")
df_train_raw['saledate'] = pd.to_datetime(df_train_raw['saledate'])
df_valid_raw['saledate'] = pd.to_datetime(df_valid_raw['saledate'])
df_train_raw = df_train_raw[df_train_raw['saledate'].dt.year >= 2008].copy()
df_valid_raw = df_valid_raw[df_valid_raw['saledate'].dt.year >= 2008].copy()
print(f"לאחר סינון, נותרו {len(df_train_raw)} רשומות בסט האימון.")


# --- שלב 2: איחוד קבצים לעיבוד אחיד ---

# !!! התיקון כאן: יצירת המשתנה train_labels *אחרי* הסינון !!!
train_labels = df_train_raw['SalePrice'].copy()
df_train_raw = df_train_raw.drop('SalePrice', axis=1)

df_train_raw['source'] = 'train'
df_valid_raw['source'] = 'valid'
df_combined = pd.concat([df_train_raw, df_valid_raw], ignore_index=True, sort=False)

# --- שלב 3: הנדסת מאפיינים ---
df_combined['saleYear'] = df_combined['saledate'].dt.year
df_combined = df_combined.drop('saledate', axis=1)
df_combined['machineAge'] = df_combined['saleYear'] - df_combined['YearMade']
valid_age_median = df_combined[df_combined['YearMade'] != 1000]['machineAge'].median()
df_combined.loc[df_combined['YearMade'] == 1000, 'machineAge'] = valid_age_median
df_combined['fiProductClassDesc'] = df_combined['fiProductClassDesc'].fillna('')
keywords_to_extract = ['excavator', 'dozer', 'loader', 'crawler', 'wheel', 'track']
for keyword in keywords_to_extract:
    df_combined[f'is_{keyword}'] = df_combined['fiProductClassDesc'].str.contains(keyword, case=False).astype(int)

# --- שלב 4: טיפול בערכים חסרים ---
numeric_cols_missing = ['MachineHoursCurrentMeter', 'auctioneerID']
for col in numeric_cols_missing:
    df_combined[col + '_is_missing'] = df_combined[col].isnull()
    median_val = df_combined[col].median()
    df_combined[col] = df_combined[col].fillna(median_val)
categorical_cols_missing = [col for col in df_combined.columns if pd.api.types.is_object_dtype(df_combined[col]) and df_combined[col].isnull().sum() > 0]
for col in categorical_cols_missing:
    df_combined[col] = df_combined[col].fillna('missing')

# --- שלב 5: המרת עמודות קטגוריאליות למספרים ---
source_col = df_combined['source']
df_combined = df_combined.drop('source', axis=1)
cols_to_drop = []
for col_name in df_combined.columns:
    if pd.api.types.is_object_dtype(df_combined[col_name]):
        num_unique_values = df_combined[col_name].nunique()
        if num_unique_values <= 5:
            dummies = pd.get_dummies(df_combined[col_name], prefix=col_name)
            df_combined = pd.concat([df_combined, dummies], axis=1)
            cols_to_drop.append(col_name)
        else:
            df_combined[col_name] = pd.Categorical(df_combined[col_name]).codes
df_combined = df_combined.drop(columns=cols_to_drop)
df_combined['source'] = source_col

# --- שלב 6: פיצול, אימון מודל והערכה (בשיטת Time-Based) ---
df_train_processed = df_combined[df_combined['source'] == 'train'].drop('source', axis=1).copy()
df_valid_processed = df_combined[df_combined['source'] == 'valid'].drop('source', axis=1).copy()
df_train_processed['SalePrice'] = train_labels.values # .values ensures correct assignment without index alignment issues

# 6.1: פיצול מבוסס-זמן לצורך הערכה מקומית
val_year = 2011
train_time_split = df_train_processed[df_train_processed['saleYear'] <= val_year - 1]
val_time_split = df_train_processed[df_train_processed['saleYear'] == val_year]
X_train_time = train_time_split.drop('SalePrice', axis=1)
y_train_time = train_time_split['SalePrice']
X_val_time = val_time_split.drop('SalePrice', axis=1)
y_val_time = val_time_split['SalePrice']

# 6.2: אימון והערכה על הפיצול המקומי
print("\n--- שלב הערכה: אימון על נתוני עבר (2008-2010) ובדיקה על 2011 ---")
model_time_split = RandomForestRegressor(n_jobs=-1, random_state=42)
model_time_split.fit(X_train_time, y_train_time)
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))
val_preds = model_time_split.predict(X_val_time)
val_rmsle_score = rmsle(y_val_time, val_preds)
print(f"ערך הפיספוס (RMSLE) על סט האימות של 2011 הוא: {val_rmsle_score:.5f}")

# --- שלב 7: אימון מודל סופי על כל נתוני האימון ---
print("\n" + "="*50 + "\n--- שלב סופי: אימון מחדש על כל נתוני האימון (2008-2011) ---")
X_full_train = df_train_processed.drop('SalePrice', axis=1)
y_full_train = df_train_processed['SalePrice']
final_model = RandomForestRegressor(n_jobs=-1, random_state=42)
final_model.fit(X_full_train, y_full_train)
print("אימון המודל הסופי הושלם.")

# --- שלב 8: יצירת קובץ הגשה על נתוני 2012 ---
print("\n" + "="*50 + "\n--- יצירת קובץ הגשה על נתוני 2012 באמצעות המודל הסופי ---")
train_cols = set(X_full_train.columns)
valid_cols = set(df_valid_processed.columns)
missing_in_valid = list(train_cols - valid_cols)
for c in missing_in_valid:
    df_valid_processed[c] = 0
df_valid_processed = df_valid_processed[X_full_train.columns]
valid_predictions = final_model.predict(df_valid_processed)
df_submission = pd.DataFrame({'SalesID': df_valid_raw['SalesID'], 'SalePrice': valid_predictions})
submission_filename = f'{DRIVE_PATH}submission.csv'
df_submission.to_csv(submission_filename, index=False)
print(f"\nקובץ ההגשה '{submission_filename}' נשמר בהצלחה ב-Google Drive!")
print(df_submission.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
מבצע סינון ראשוני לשמירת נתונים משנת 2008 ואילך...
לאחר סינון, נותרו 152203 רשומות בסט האימון.

--- שלב הערכה: אימון על נתוני עבר (2008-2010) ובדיקה על 2011 ---
ערך הפיספוס (RMSLE) על סט האימות של 2011 הוא: 0.25515

--- שלב סופי: אימון מחדש על כל נתוני האימון (2008-2011) ---
אימון המודל הסופי הושלם.

--- יצירת קובץ הגשה על נתוני 2012 באמצעות המודל הסופי ---

קובץ ההגשה '/content/drive/MyDrive/KaggleProject/submission.csv' נשמר בהצלחה ב-Google Drive!
   SalesID  SalePrice
0  1222837    45580.0
1  1222839    77070.0
2  1222841    35510.0
3  1222843    17065.0
4  1222845    39230.0


In [3]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# 6.2: הגדרת רשת הפרמטרים לחיפוש אקראי
# אלו טווחים רחבים שמהם האלגוריתם ידגום באופן אקראי
random_grid = {
    'n_estimators': [int(x) for x in np.linspace(start=100, stop=1000, num=10)],
    'max_features': ['sqrt', 'log2', 1.0], # 1.0 = auto
    'max_depth': [int(x) for x in np.linspace(10, 110, num=11)] + [None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 6.3: יצירה והרצה של החיפוש האקראי
print("\n--- מתחיל חיפוש אקראי של היפר-פרמטרים... (זה עשוי לקחת זמן) ---")
rf = RandomForestRegressor(random_state=42)

# n_iter=50 : ננסה 50 צירופים אקראיים
# cv=3 : נשתמש באימות צולב עם 3 פולים
rs = RandomizedSearchCV(estimator=rf,
                          param_distributions=random_grid,
                          n_iter=20,
                          cv=3,
                          verbose=2,
                          random_state=42,
                          n_jobs=4)

# הרצת החיפוש על נתוני האימון מבוססי-הזמן
rs.fit(X_train_time, y_train_time)

# 6.4: הצגת הפרמטרים הטובים ביותר שנמצאו
print("\nהפרמטרים הטובים ביותר שנמצאו בחיפוש האקראי:")
print(rs.best_params_)

# 6.5: הערכת המודל עם הפרמטרים הטובים ביותר
best_model_random = rs.best_estimator_
val_preds = best_model_random.predict(X_val_time)
val_rmsle_score = rmsle(y_val_time, val_preds)
print(f"\nערך הפיספוס (RMSLE) עם המודל המכוונן הוא: {val_rmsle_score:.5f}")

# --- שלב 7: אימון מודל סופי על כל נתוני האימון עם הפרמטרים הטובים ביותר ---
print("\n" + "="*50 + "\n--- שלב סופי: אימון מחדש על כל נתוני האימון עם הפרמטרים המיטביים ---")
X_full_train = df_train_processed.drop('SalePrice', axis=1)
y_full_train = df_train_processed['SalePrice']

# יצירת מודל חדש עם הפרמטרים הטובים ביותר שמצאנו
final_model = RandomForestRegressor(n_jobs=-1,
                                    random_state=42,
                                    **rs.best_params_) # <-- שימוש בפרמטרים הטובים ביותר

final_model.fit(X_full_train, y_full_train)
print("אימון המודל הסופי והמכוונן הושלם.")

# --- שלב 8: יצירת קובץ הגשה על נתוני 2012 ---
print("\n" + "="*50 + "\n--- יצירת קובץ הגשה על נתוני 2012 באמצעות המודל הסופי ---")
train_cols = set(X_full_train.columns)
valid_cols = set(df_valid_processed.columns)
missing_in_valid = list(train_cols - valid_cols)
for c in missing_in_valid:
    df_valid_processed[c] = 0
df_valid_processed = df_valid_processed[X_full_train.columns]
valid_predictions = final_model.predict(df_valid_processed)
df_submission = pd.DataFrame({'SalesID': df_valid_raw['SalesID'], 'SalePrice': valid_predictions})
submission_filename = f'{DRIVE_PATH}submission.csv'
df_submission.to_csv(submission_filename, index=False)
print(f"\nקובץ ההגשה '{submission_filename}' נשמר בהצלחה ב-Google Drive!")
print(df_submission.head())




--- מתחיל חיפוש אקראי של היפר-פרמטרים... (זה עשוי לקחת זמן) ---
Fitting 3 folds for each of 20 candidates, totalling 60 fits
